In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mlp
import pickle
import os
from matplotlib import pyplot as plt


In [ ]:
sign_name_carolo_trans = {
    1: 0,
    9: 1,
    12: 2,
    13: 3,
    14: 4,
    33: 5,
    34: 6,
    41: 7,
    43: 20,
    100: 8,
    101: 9,
    102: 10,
    103: 11,
    104: 12,
    105: 13,
    106: 14,
    107: 15,
    108: 16,
    109: 17,
    110: 18,
    111: 19
}

In [ ]:
sign_name_carolo_dict_new = {
    0: '30 Zone Anfang (speed limit start)',
    1: 'Absolutes Überholverbot Anfang (no passing zone start)',
    2: 'Vorfahrtstraße (priority on next intersections)',
    3: 'Vorfahrt Gewähren (give way to incoming)',
    4: 'Stop Zeichen (stop for at least 3 sec)',
    5: 'Vorgeschriebene Fahrtrichtung Rechts (turn right on intersection)',
    6: 'Vorgeschriebene Fahrtrichtung Links (turn left on intersection)',
    7: 'Absolutes Überholverbot Ende (no passing zone end)',
    8: 'Gegenverkehr Vorrang gewähren (Barred area, let oncoming pass)',
    9: '30 Zone Ende (end of speed limit)',
    10: 'Fußgängerüberweg (crosswalk)',
    11: 'Richtungstafel Kurve links (sharp turn left)',
    12: 'Richtungstafel Kurve links (even sharper turn left)',
    13: 'Richtungstafel Kurve rechts (sharp turn right)',
    14: 'Richtungstafel Kurve rechts (even sharper turn right)',
    15: 'Gefälle 10% (downhill 10%)',
    16: 'Steigung 10% (uphill 10%)',
    17: 'Parken Bereich (parking zone)',
    18: 'Kraftfahrtstraße Anfang (expressway begin)',
    19: 'Kraftfahrtstraße Ende (expressway end)'
}

In [ ]:
def transform_coordinates(x1,y1,x2,y2,dimX,dimY):
    centerX=(x1+x2)/2
    centerY=(y1+y2)/2
    width=x2-x1
    height=y2-y1
    #to relative values:
    centerX=centerX/dimX
    centerY=centerY/dimY
    width=width/dimX
    height=height/dimY
    
    return centerX,centerY,width,height
    

# Creating labelfiles

In [ ]:
frames = []
datafiles=[]
files=[]
oldFile=None
num_Zero=2000
counter=0
basepath='/data/Images/'
#datafiles.append(basepath+'test_environment')
datafiles.append(basepath+'signs001_manually_labeled')
datafiles.append(basepath+'signs003_manually_labeled')
datafiles.append(basepath+'signs004_manually_labeled')
datafiles.append(basepath+'signs006_bosch_manually_labeled')
for folder in datafiles:
    for filename in os.listdir(folder):
        if filename.endswith('.csv'):
            print(filename)
            df = pd.read_csv(folder+'/'+filename)
            df['ClassId'].replace(sign_name_carolo_trans, inplace=True)
            for index, row in df.iterrows():           
                classid=row['ClassId']
                centerX,centerY,width,height=transform_coordinates(row['Roi.X1'],row['Roi.Y1'],row['Roi.X2'],row['Roi.Y2'],row['Width'],row['Height'])
                file=row['Filename'].split('.')[0]
                if classid!=20:
                    if file==oldFile:
                        #print(centerX,centerY,width,height)
                        f = open(folder+'/'+file+'.txt',"a") 
                        f.write(str(classid)+' '+str(centerX)+' '+str(centerY)+' '+str(width)+' '+str(height)+'\n')
                        f.close()
                    else:
                        f = open(folder+'/'+file+'.txt',"w") 
                        f.write(str(classid)+' '+str(centerX)+' '+str(centerY)+' '+str(width)+' '+str(height)+'\n')
                        f.close()
                        #print(folder+'/'+file)
                        files.append(folder+'/'+file)
                elif classid==20 and counter<num_Zero:
                    f = open(folder+'/'+file+'.txt',"w") 
                    f.close()
                    counter+=1
                    files.append(folder+'/'+file)
                oldFile=file
                
                #frames_added_april.append(pd.read_csv(filename))

# Train & Test files

In [ ]:
#from sklearn.model_selection import train_test_split

In [ ]:
#X_train,X_test=train_test_split(files, shuffle=False)

num_of_rows = round((len(files)) * 0.8)
files_np=np.array(files)
np.random.shuffle(files_np) 
X_train= files_np[:num_of_rows] 
X_test = files_np[num_of_rows:] 

file = open(basepath+'train.txt',"w")
for line in X_train:
    file.write(str(line)+'.png\n')
file.close()
    
file = open(basepath+'test.txt',"w")
for line in X_test:
    file.write(str(line)+'.png\n')
file.close()
    

In [ ]:
with open('../sign_recognition/dict/sign_names_dict.pkl', 'rb') as f:
    sign_names = pickle.load(f)

In [ ]:
sign_names